In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import ml
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

Drop cols, get rid of NaN (1-replace, 2-drop), convert objects to numbers

In [ ]:
df = pd.read_csv("train.csv", sep=",")
dftest = pd.read_csv("test.csv", sep=",")

df.drop(['Name', 'Cabin'], axis=1, inplace=True)
dftest.drop(['Name', 'Cabin'], axis=1, inplace=True)

for i in range(df.shape[1]-1):
    s = df.iloc[:, i]
    stest = dftest.iloc[:, i]
    if is_numeric_dtype(s):       
        s.fillna(s.mean(), inplace=True)
        stest.fillna(0, inplace=True)

df.dropna(axis=0, inplace=True)

tofact = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Transported']

for name in tofact:
    df[name] = df[name].factorize()[0]
    if name != 'Transported':
        dftest[name] = dftest[name].factorize()[0]
    
dftest

Get non-discrete data

In [ ]:
tempf = pd.DataFrame(df.loc[:, 'RoomService':'Transported'])
tempf

Show correlation

In [ ]:
for name in ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']:
    x = []
    y = []
    for low in np.arange(0, tempf[name].max()+tempf[name].mean(), tempf[name].mean()*20):
        high = low+tempf[name].mean()
        x.append((high+low)/2)
        y.append(tempf[(tempf[name] >= low) & 
                    (tempf[name] < high) & 
                    (tempf['Transported'] == True)].count()[0])
        plt.plot(x,y)

plt.show()

Separate to test and train

In [ ]:
df = df.sample(frac=1)
x_train = df.iloc[0 : int(df.shape[0]*0.8)]
y_train = x_train.loc[:, 'Transported']
x_test = df.iloc[int(df.shape[0]*0.8) : df.shape[0]]
y_test = x_test.loc[:, 'Transported']

y_train = y_train.astype(float)
y_test = y_test.astype(float)

Make Neural Network

In [ ]:
nets = []
for i in range(10):
    nets.append(ml.NeuralNet([ml.Dense(x_train.shape[1]-2, 
                                        activation=ml.Sigmoid(), 
                                        weight_init="glorot", 
                                        seed=np.random.randint(10000,100000), dropout=1),

                                ml.Dense(1, activation=ml.Sigmoid(), 
                                            weight_init="glorot", 
                                            dropout=1.0, 
                                            seed=np.random.randint(0,10000))],
                                            
                                loss=ml.MeanSquaredError()))

optimizer = ml.SGDMomentum(0.0065, 0.0005, 1, "exp")

for i, net in enumerate(nets):
    trainer = ml.Trainer(net, optimizer)
    trainer.train(x_train.iloc[:, 1:x_train.shape[1]-1].values,
                            y_train.values.reshape(y_train.values.shape[0], 1),
                            x_test.iloc[:, 1:x_test.shape[1]-1].values,
                            y_test.values.reshape(y_test.values.shape[0], 1),
                            10, (i+1)*10, 1)
            
    print(trainer._best_loss)



Save predictions

In [ ]:
res = pd.DataFrame()
for i, net in enumerate(nets):    
    res[i]= net.forward(dftest.iloc[:, 1:]).squeeze()

res = res.apply(lambda x: x > x.mean())

res2 = pd.DataFrame()
res2['PassengerId'] = dftest.PassengerId
s = []
for i in range(res.shape[0]):
    s.append(res.iloc[i].describe()['top'])

s = pd.Series(s)
res2['Transported'] = s


res2.to_csv("submission.csv", index=False)